In [103]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from subprocess import check_output

Pipeline at a glance:
load data -> cleaning/preprocessing -> feature eng -> format data for modelling -> fit model -> evaluate model -> generate submission file

Step 1: Load data

In [104]:
df_train = pd.read_csv('../input/train.csv')
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [105]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [106]:
df_test = pd.read_csv('../input/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [107]:
df_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## Step 2: Basic cleaning and preprocessing

In [108]:
# label encode sex
le = LabelEncoder()
le.fit(df_train.Sex)
df_train.loc[:,'Sex'] = le.transform(df_train.Sex)
df_test.loc[:,'Sex'] = le.transform(df_test.Sex)


In [109]:
# Fill in the missing values
mAge = pd.concat((df_train.Age, df_test.Age), axis=0).mean()
df_train.loc[:, 'Age'] = df_train.Age.fillna(mAge)
df_test.loc[:, 'Age'] = df_test.Age.fillna(mAge)

# Step 3: Feature engineering

In [110]:
df_train['Age_binned'] = pd.cut(df_test['Age'], [0,16,32,48,64,200], labels = [0,1,2,3,4], retbins=False)
df_test['Age_binned'] = pd.cut(df_test['Age'], [0,16,32,48,64,200], labels = [0,1,2,3,4], retbins=False)

In [111]:
df_test['Embarked'].fillna('S',inplace=True)
df_train['Embarked'].fillna('S',inplace=True)

In [112]:
df_train['Family_size'] = df_train['SibSp'] + df_train['Parch']
df_test['Family_size'] = df_test['SibSp'] + df_test['Parch']

In [113]:
df_train['Is_Alone'] = (df_train['Family_size'] == 0).astype(int)
df_train['Is_Alone'].drop

df_test['Is_Alone'] = (df_test['Family_size'] == 0).astype(int)
df_test['Is_Alone'].drop

<bound method Series.drop of 0      1
1      0
2      1
3      1
4      0
5      1
6      1
7      0
8      1
9      0
10     1
11     1
12     0
13     0
14     0
15     0
16     1
17     1
18     0
19     1
20     0
21     0
22     1
23     0
24     0
25     0
26     0
27     1
28     1
29     0
      ..
388    1
389    0
390    1
391    0
392    0
393    1
394    0
395    0
396    1
397    0
398    1
399    1
400    1
401    0
402    0
403    1
404    0
405    1
406    0
407    0
408    1
409    0
410    1
411    0
412    1
413    1
414    1
415    1
416    1
417    0
Name: Is_Alone, Length: 418, dtype: int64>

In [114]:
le.fit(df_train.Embarked)
df_train.loc[:,'Embarked'] = le.transform(df_train.Embarked)
df_test.loc[:,'Embarked'] = le.transform(df_test.Embarked)

In [120]:
df_train = df_train.drop(columns = ['PassengerId','Name','Age','SibSp','Parch','Ticket','Fare','Cabin'], axis=1, inplace = False)
df_test = df_test.drop(columns = ['PassengerId','Name','Age','SibSp','Parch','Ticket','Fare','Cabin'], axis=1, inplace = False)

KeyError: "['PassengerId' 'Name' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare' 'Cabin'] not found in axis"

In [121]:
df_train

,Survived,Pclass,Sex,Embarked,Age_binned,Family_size,Is_Alone
0,0,3,1,2,2.0,1,0
1,1,1,0,0,2.0,1,0
2,1,3,0,2,3.0,0,1
3,1,1,0,2,1.0,1,0
4,0,3,1,2,1.0,0,1
5,0,3,1,1,0.0,0,1
6,0,1,1,2,1.0,0,1
7,0,3,1,2,1.0,4,0
8,1,3,0,2,1.0,2,0
9,1,2,0,0,1.0,1,0


In [117]:
from sklearn.preprocessing import OneHotEncoder

In [119]:

myEncoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
myEncoder.fit(df_train['Pclass','Sex','Embarked','Age_binned','Is_Alone'])



KeyError: ('Pclass', 'Sex', 'Embarked', 'Age_binned', 'Is_Alone')

In [19]:
pd.concat([df_train.drop('Survived', 'Pclass','Sex','Embarked','Age_binned','Is_Alone', 1),
          pd.DataFrame(myEncoder.transform(df_train[columnsToEncode]))], axis=1).reindex()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_binned,Family_size,Is_Alone
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,S,2.0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,C,2.0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,3.0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,S,1.0,1,0
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,S,1.0,0,1
5,6,0,3,"Moran, Mr. James",1,29.881138,0,0,330877,8.4583,NaN,Q,0.0,0,1
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,17463,51.8625,E46,S,1.0,0,1
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,349909,21.0750,NaN,S,1.0,4,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,347742,11.1333,NaN,S,1.0,2,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,237736,30.0708,NaN,C,1.0,1,0


<bound method Series.drop of 0      0
1      0
2      1
3      0
4      1
5      1
6      1
7      0
8      0
9      0
10     0
11     1
12     1
13     0
14     1
15     1
16     0
17     1
18     0
19     1
20     1
21     1
22     1
23     1
24     0
25     0
26     1
27     0
28     1
29     1
      ..
861    0
862    1
863    0
864    1
865    1
866    0
867    1
868    1
869    0
870    1
871    0
872    1
873    1
874    0
875    1
876    1
877    1
878    1
879    0
880    0
881    1
882    1
883    1
884    1
885    0
886    1
887    1
888    0
889    1
890    1
Name: Is_Alone, Length: 891, dtype: int64>

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_binned,Family_size,Is_Alone
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,S,2.0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,C,2.0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,S,3.0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,S,1.0,1,0
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,S,1.0,0,1
5,6,0,3,"Moran, Mr. James",1,29.881138,0,0,330877,8.4583,NaN,Q,0.0,0,1
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,17463,51.8625,E46,S,1.0,0,1
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,349909,21.0750,NaN,S,1.0,4,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,347742,11.1333,NaN,S,1.0,2,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,237736,30.0708,NaN,C,1.0,1,0


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_binned,Family_size,Is_Alone
0,1,0,3,"Braund, Mr. Owen Harris",1,22.000000,1,0,A/5 21171,7.2500,NaN,2,2.0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.000000,1,0,PC 17599,71.2833,C85,0,2.0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.000000,0,0,STON/O2. 3101282,7.9250,NaN,2,3.0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.000000,1,0,113803,53.1000,C123,2,1.0,1,0
4,5,0,3,"Allen, Mr. William Henry",1,35.000000,0,0,373450,8.0500,NaN,2,1.0,0,1
5,6,0,3,"Moran, Mr. James",1,29.881138,0,0,330877,8.4583,NaN,1,0.0,0,1
6,7,0,1,"McCarthy, Mr. Timothy J",1,54.000000,0,0,17463,51.8625,E46,2,1.0,0,1
7,8,0,3,"Palsson, Master. Gosta Leonard",1,2.000000,3,1,349909,21.0750,NaN,2,1.0,4,0
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",0,27.000000,0,2,347742,11.1333,NaN,2,1.0,2,0
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",0,14.000000,1,0,237736,30.0708,NaN,0,1.0,1,0


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_binned,Family_size
0,892,3,"Kelly, Mr. James",1,34.500000,0,0,330911,7.8292,NaN,1,2,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.000000,1,0,363272,7.0000,NaN,2,2,1
2,894,2,"Myles, Mr. Thomas Francis",1,62.000000,0,0,240276,9.6875,NaN,1,3,0
3,895,3,"Wirz, Mr. Albert",1,27.000000,0,0,315154,8.6625,NaN,2,1,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22.000000,1,1,3101298,12.2875,NaN,2,1,2
5,897,3,"Svensson, Mr. Johan Cervin",1,14.000000,0,0,7538,9.2250,NaN,2,0,0
6,898,3,"Connolly, Miss. Kate",0,30.000000,0,0,330972,7.6292,NaN,1,1,0
7,899,2,"Caldwell, Mr. Albert Francis",1,26.000000,1,1,248738,29.0000,NaN,2,1,2
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",0,18.000000,0,0,2657,7.2292,NaN,0,1,0
9,901,3,"Davies, Mr. John Samuel",1,21.000000,2,0,A/4 48871,24.1500,NaN,2,1,2


## Step 4: Format data for modelling

In [11]:
# Split data into X and y, and select features to use
X = df_train.loc[:,['Age', 'Sex', 'Pclass']]
y = df_train.loc[:,'Survived']

In [12]:
# Split data X and y into train and val sets
X_test = df_test.loc[:,['Age', 'Sex', 'Pclass']]
X_train, X_valid, y_train, y_valid = train_test_split(
      X, y, test_size=0.2, random_state=42)

## Step 5: Train (or fit) the model

In [13]:
# Fit logistic regression
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

## Step 6: Evaluate the model

In [14]:
# define a scoring function
def acc(y: np.array, y_pred: np.array) -> float:
    return np.sum(y_pred==y)/len(y)

In [15]:
# Use model to predict validation set
y_pred_valid = LR.predict(X_valid)

In [16]:
print('Validation set accuracy', acc(y_valid, y_pred_valid), '%')

Validation set accuracy 0.7877094972067039 %


## Step 7: Generate Submission

In [17]:
# Predict for test set

y_pred_test = LR.predict(X_test)

# Create a Kaggle submission
sub = pd.DataFrame({'PassengerId': df_test['PassengerId'],
                    'Survived': y_pred_test})

sub.to_csv('basic_submission.csv', index=False)